In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='ticks')

from lab.classes.dbclasses import dbMouse, dbExperiment
from lab.classes import ExperimentGroup

import numpy as np

from scipy.stats import wilcoxon, ttest_ind

import pandas as pd
import lab.analysis.behavior_analysis as ba

In [2]:
%run /home/sebi/code/analysis/analysis-scripts/induction_analysis/group_scripts/init_all.py

/home/sebi/code/analysis/analysis-scripts/induction_analysis/Analysis/induction_analysis.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipympl.backend_nbagg' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/sebi/.local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sebi/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sebi/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  

[(0, 'SCE'), (1, 'Sparse Targeted'), (2, 'Dense'), (3, 'Dense + Dreadd'), (4, 'Sparse LED')]
Vars in Environment: grps, labels, colors, stim_filters, ind_filters


In [3]:
dense_grp = grps[2]
cno_grp = grps[3]

In [76]:
frs = []

for grp in [dense_grp, cno_grp]:
    
    prefix = grp[0].session.split('_')[0]

    for sess in ['_baseline', '_induction', '_24h']:
        newgrp = [x._get_session(prefix + sess) for x in grp]
        for expt in newgrp:
            
            if expt is None:
                continue

#             running = expt.runningIntervals(returnBoolList=True)[0]
            if sess == '_induction':
                abs_pos = ba.absolutePosition(grp[0].find('trial'))
                first_idx = np.where(abs_pos > 6)[0][0]
                running = np.where(expt.velocity()[0][first_idx:] > 1)[0] + first_idx
                nonrunning = np.where(expt.velocity()[0][first_idx:] <= 1)[0] + first_idx
            else:
                running = np.where(expt.velocity()[0] > 1)[0]
                nonrunning = np.where(expt.velocity()[0] <= 1)[0]
        
            spikes = expt.spikes(binary=True)
            
            running_fr = np.nansum(spikes[:, running], axis=1) / (len(running) * expt.frame_period())
            nonrunning_fr = np.nansum(spikes[:, nonrunning], axis=1) / (len(nonrunning) * expt.frame_period())
            
            full_fr = np.nansum(spikes, axis=1) / (spikes.shape[1] * expt.frame_period())

            frs.append({'mouse_name': expt.parent.mouse_name,
                        'session': prefix + sess,
                        'condition': prefix,
                        'sess': sess,
                        'run_value': np.nanmean(running_fr),
                        'nonrun_value': np.nanmean(nonrunning_fr),
                        'value': np.nanmean(full_fr)})

frs = pd.DataFrame(frs)

control_24h not in association dictionary {u'control': {u'pre': 22349, u'post': 22341, u'induction': 22286, u'baseline': 22287}, u'cno': {u'pre': 22344, u'post': 22348, u'induction': 22326, u'baseline': 22327}}
control_24h not in association dictionary {u'control': {u'pre': 22346, u'post': 22340, u'induction': 22311, u'baseline': 22313}, u'cno': {u'pre': 22345, u'post': 22350, u'induction': 22318, u'baseline': 22317}}
control_24h not in association dictionary {u'control': {u'pre': 24601, u'induction': 24603, u'baseline': 24602}, u'cno': {}}
cno_24h not in association dictionary {u'control': {u'pre': 22349, u'post': 22341, u'induction': 22286, u'baseline': 22287}, u'cno': {u'pre': 22344, u'post': 22348, u'induction': 22326, u'baseline': 22327}}
cno_24h not in association dictionary {u'control': {u'pre': 22346, u'post': 22340, u'induction': 22311, u'baseline': 22313}, u'cno': {u'pre': 22345, u'post': 22350, u'induction': 22318, u'baseline': 22317}}
cno_24h not in association dictionary {

In [56]:
def formatPlot(ax, offset=10):
    for axis in ['left', 'right', 'bottom', 'top']:
        ax.spines[axis].set_linewidth(2)
    ax.tick_params(labelsize=14, width=2, length=4)
    sns.despine(ax=ax, offset=offset)
    ax.set_xlabel('')
    ax.set_ylabel('')

In [86]:
fig = plt.figure()
ax = fig.add_subplot(111)

colors = ['warm blue', 'golden rod']
colors = sns.xkcd_palette(colors)

sns.boxplot(x='sess', y='value', hue='condition', data=frs, ax=ax,
            whis=False, showcaps=False, showfliers=False)
plt.setp(ax.artists, edgecolor='k', facecolor='w')
plt.setp(ax.lines, color='k')

sns.swarmplot(x='sess', y='value', hue='condition', dodge=True, data=frs, palette=colors,
              ax=ax, size=7, edgecolor='k', linewidth=1)

formatPlot(ax)

ax.set_xticklabels(['PRE', 'POST', '24H', 'PRE', 'POST', '24H'])
ax.set_ylabel('Mean Event Rate(Hz)', fontsize=18)

ax.legend().set_visible(False)

fig.savefig('/home/sebi/plots/ind/figs4/cno_mean_fr.svg', bbox_inches='tight')

FigureCanvasNbAgg()

In [81]:
print ttest_ind(frs.loc[frs['session'] == 'control_baseline']['value'], frs.loc[frs['session'] == 'cno_baseline']['value'])
print ttest_ind(frs.loc[frs['session'] == 'control_induction']['value'], frs.loc[frs['session'] == 'cno_induction']['value'])
print ttest_ind(frs.loc[frs['session'] == 'control_24h']['value'], frs.loc[frs['session'] == 'cno_24h']['value'])

Ttest_indResult(statistic=-2.6301515829184243, pvalue=0.015645895416039283)
Ttest_indResult(statistic=-3.082171093075783, pvalue=0.005650248279354123)
Ttest_indResult(statistic=-1.313844718706283, pvalue=0.2086385374982976)


In [83]:
frs.groupby(['session']).std()

,nonrun_value,run_value,value
session,,,
cno_24h,0.034700,0.043333,0.039515
cno_baseline,0.061748,0.087301,0.067611
cno_induction,0.050138,0.058661,0.050965
control_24h,0.030879,0.034112,0.032045
control_baseline,0.039060,0.051016,0.044507
control_induction,0.030854,0.044073,0.036099


In [82]:
frs.groupby(['session']).mean()

,nonrun_value,run_value,value
session,,,
cno_24h,0.078598,0.102507,0.093209
cno_baseline,0.133548,0.149249,0.138632
cno_induction,0.137852,0.115129,0.134692
control_24h,0.060052,0.077010,0.070402
control_baseline,0.066419,0.083485,0.076386
control_induction,0.066923,0.083146,0.078325


In [72]:
fig = plt.figure()
ax = fig.add_subplot(111)

colors = ['warm blue', 'golden rod']
colors = sns.xkcd_palette(colors)

sns.boxplot(x='sess', y='run_value', hue='condition', data=frs, ax=ax,
            whis=False, showcaps=False, showfliers=False)
plt.setp(ax.artists, edgecolor='k', facecolor='w')
plt.setp(ax.lines, color='k')

sns.swarmplot(x='sess', y='run_value', hue='condition', dodge=True, data=frs, palette=colors,
              ax=ax, size=7, edgecolor='k', linewidth=1)

formatPlot(ax)

ax.set_xticklabels(['PRE', 'POST', '24H', 'PRE', 'POST', '24H'])
ax.set_ylabel('Mean Event Rate during Running (Hz)', fontsize=18)

ax.legend().set_visible(False)

ax.set_ylim([0, 0.4])

FigureCanvasNbAgg()

(0, 0.4)

In [40]:
from scipy.stats import ttest_ind

In [65]:
print ttest_ind(frs.loc[frs['session'] == 'control_baseline']['run_value'], frs.loc[frs['session'] == 'cno_baseline']['run_value'])
print ttest_ind(frs.loc[frs['session'] == 'control_induction']['run_value'], frs.loc[frs['session'] == 'cno_induction']['run_value'])
print ttest_ind(frs.loc[frs['session'] == 'control_24h']['run_value'], frs.loc[frs['session'] == 'cno_24h']['run_value'])

Ttest_indResult(statistic=-2.2297116809965516, pvalue=0.036812893664160004)
Ttest_indResult(statistic=-1.4867277652120807, pvalue=0.15194754522441623)
Ttest_indResult(statistic=-1.3562885466715637, pvalue=0.19507268924652246)


In [73]:
fig = plt.figure()
ax = fig.add_subplot(111)

colors = ['warm blue', 'golden rod']
colors = sns.xkcd_palette(colors)

sns.boxplot(x='sess', y='nonrun_value', hue='condition', data=frs, ax=ax,
            whis=False, showcaps=False, showfliers=False)
plt.setp(ax.artists, edgecolor='k', facecolor='w')
plt.setp(ax.lines, color='k')

sns.swarmplot(x='sess', y='nonrun_value', data=frs, hue='condition', dodge=True, palette=colors,
              ax=ax, size=7, edgecolor='k', linewidth=1)

formatPlot(ax)

ax.set_xticklabels(['PRE', 'POST', '24H', 'PRE', 'POST', '24H'])
ax.set_ylabel('Mean Event Rate during Immobility (Hz)', fontsize=18)

ax.legend().set_visible(False)
ax.set_ylim([0, 0.4])

FigureCanvasNbAgg()

(0, 0.4)

In [74]:
print ttest_ind(frs.loc[frs['session'] == 'control_baseline']['nonrun_value'], frs.loc[frs['session'] == 'cno_baseline']['nonrun_value'])
print ttest_ind(frs.loc[frs['session'] == 'control_induction']['nonrun_value'], frs.loc[frs['session'] == 'cno_induction']['nonrun_value'])
print ttest_ind(frs.loc[frs['session'] == 'control_24h']['nonrun_value'], frs.loc[frs['session'] == 'cno_24h']['nonrun_value'])

Ttest_indResult(statistic=-3.1449769621109773, pvalue=0.004888381433521167)
Ttest_indResult(statistic=-4.126446805393002, pvalue=0.0004802333469860523)
Ttest_indResult(statistic=-1.1665793757300937, pvalue=0.2615930279599925)


In [89]:
cno_grp[0].get('tSeriesDirectory')

'/data/Mohsin/2p/msa0919_2/10_10_19/TSeriesmsa0919_2_10102019_cno_induce-003'

In [88]:
dense_grp[0].get('tSeriesDirectory')

'/data/Mohsin/2p/msa0919_2/10_3_19/TSeriesmsa0919_2_10032019_induce-003'

In [90]:
bsigs = dense_grp[0]._get_session('control_baseline').imagingData(dFOverF='from_file')[..., 0]
csigs = dense_grp[0]._get_session('cno_baseline').imagingData(dFOverF='from_file')[..., 0]

In [107]:
bspikes = dense_grp[0]._get_session('control_baseline').spikes()
cspikes = dense_grp[0]._get_session('cno_baseline').spikes()

In [99]:
from scipy.ndimage.filters import gaussian_filter1d

In [104]:
def norm_sig(sigs, spikes):
    
    out = []
    
    for sig, spike in zip(sigs, spikes):
        
        noise = np.where(spike == 0)
        smean = np.nanmean(sig[noise])
        sstd = np.nanstd(sig[noise])
        
        out.append((sig - smean) / sstd)
    
    return np.vstack(out)

In [117]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(gaussian_filter1d(bmean, 15) + 2 * i , color=colors[0], alpha=0.75)
ax.plot(gaussian_filter1d(cmean, 15) + 2 * i, color=colors[1], alpha=0.75)

FigureCanvasNbAgg()

In [121]:
import itertools as it

In [127]:
fig = plt.figure()
ax = fig.add_subplot(111)

for i, bmean, cmean in zip(it.count(), bmeans, cmeans):


# bmeans = []
# cmeans = []

# for i in xrange(3):

#     bsigs = dense_grp[i]._get_session('control_baseline').imagingData(dFOverF='from_file')[..., 0]
#     csigs = dense_grp[i]._get_session('cno_baseline').imagingData(dFOverF='from_file')[..., 0]

#     bspikes = dense_grp[0]._get_session('control_baseline').spikes()
#     cspikes = dense_grp[0]._get_session('cno_baseline').spikes()

#     bmean = norm_sig(bsigs, bspikes)
#     bmean = np.nanmean(bmean, axis=0)

#     cmean = norm_sig(csigs, cspikes)
#     cmean = np.nanmean(cmean, axis=0)
    
#     bmeans.append(bmean)
#     cmeans.append(cmean)

    ax.plot(gaussian_filter1d(bmean, 15) + i, color=colors[0])
    ax.plot(gaussian_filter1d(cmean, 15) + i + 0.3, color=colors[1])

# for i in xrange(10, 20):
    
#     ax.plot(bsigs[i, :] + i)

FigureCanvasNbAgg()

In [128]:
bmeans = []
for expt in dense_grp:
    
    bsession = expt._get_session('control_baseline')
    bsigs = bsession.imagingData(dFOverF='from_file')[..., 0]
    bspikes = bsession.spikes()
    
    bmean = norm_sig(bsigs, bspikes)
    bmean = np.nanmean(bmean, axis=0)
    
    bmeans.append(bmean)

In [131]:
cmeans = []
for expt in cno_grp:
    
    bsession = expt._get_session('cno_baseline')
    bsigs = bsession.imagingData(dFOverF='from_file')[..., 0]
    bspikes = bsession.spikes()
    
    bmean = norm_sig(bsigs, bspikes)
    bmean = np.nanmean(bmean, axis=0)
    
    cmeans.append(bmean)

In [142]:
fig = plt.figure()
ax = fig.add_subplot(111)

for i in xrange(10):
    
    ax.plot(bmeans[i][27*60:27*300] + 2*i, color=colors[0])
    ax.plot(cmeans[i][27*60:27*300] + 2*i + 1, color=colors[1])

FigureCanvasNbAgg()

In [199]:
# roi_filter = grp.pcs_filter()
dense_baseline = pcExperimentGroup([x._get_session('control_baseline') for x in dense_grp], **dense_grp.args)

data = dense_baseline.data(roi_filter=None)
all_data = []
all_centroids = []
all_stim_idx = []
count = 0
for expt in dense_baseline:
#     rois = expt.rois(roi_filter=roi_filter)
#     stim_idx = [i + count for i, r in enumerate(rois) if stim_filter(r)]
#     all_stim_idx.extend(stim_idx)
#     count += len(rois)
    
    expt_data = data[expt]
    stim_loc = int(expt._get_session('control_induction')._get_stim_positions(units='normalized')[0])
    expt_data = np.roll(expt_data, 50 - stim_loc, axis=1)
#     expt_data = expt_data / np.nanmax(expt_data, axis=1, keepdims=True)
#     centroids = calc_activity_centroids(expt_data)
    centroids = np.argmax(expt_data, axis=1)
    all_data.append(expt_data)
    all_centroids.extend(centroids)

all_data = np.vstack(all_data)
sort_idx = np.argsort(all_centroids)
# all_stim_idx = np.array(all_stim_idx)
# nostim_idx = np.array([x for x in range(count) if x not in all_stim_idx])

dense_sorted_data = all_data[sort_idx, :]

In [200]:
# roi_filter = grp.pcs_filter()
cno_baseline = pcExperimentGroup([x._get_session('cno_baseline') for x in cno_grp], **cno_grp.args)

data = cno_baseline.data(roi_filter=None)
all_data = []
all_centroids = []
all_stim_idx = []
count = 0
for expt in cno_baseline:
#     rois = expt.rois(roi_filter=roi_filter)
#     stim_idx = [i + count for i, r in enumerate(rois) if stim_filter(r)]
#     all_stim_idx.extend(stim_idx)
#     count += len(rois)
    
    expt_data = data[expt]
    stim_loc = int(expt._get_session('cno_induction')._get_stim_positions(units='normalized')[0])
    expt_data = np.roll(expt_data, 50 - stim_loc, axis=1)
#     expt_data = expt_data / np.nanmax(expt_data, axis=1, keepdims=True)
#     centroids = calc_activity_centroids(expt_data)
    centroids = np.argmax(expt_data, axis=1)
    all_data.append(expt_data)
    all_centroids.extend(centroids)

all_data = np.vstack(all_data)
sort_idx = np.argsort(all_centroids)
# all_stim_idx = np.array(all_stim_idx)
# nostim_idx = np.array([x for x in range(count) if x not in all_stim_idx])

sorted_data = all_data[sort_idx, :]

In [203]:
c = sns.xkcd_rgb['black']
cmap=sns.light_palette(c, as_cmap=True, reverse=True)

In [218]:
all_vals = np.vstack([sorted_data, dense_sorted_data])
vmax = np.nanpercentile(all_vals, 95)

In [221]:
fig = plt.figure()
ax = fig.add_subplot(121)
sns.heatmap(dense_sorted_data, cmap=cmap, ax=ax, cbar=False, vmax=vmax)

n_cells = dense_sorted_data.shape[0]
ax.set_xticks([])
ax.set_yticks([])
print n_cells
# ax.set_yticks([n_cells - 1])
# ax.set_yticklabels([n_cells])

# ax.set_title('CNO-', fontsize=18)

ax = fig.add_subplot(122)
sns.heatmap(sorted_data, cmap=cmap, ax=ax, cbar=True, vmax=vmax)

n_cells = sorted_data.shape[0]
ax.set_xticks([])
ax.set_yticks([])
print n_cells
# ax.set_yticks([n_cells - 1])
# ax.set_yticklabels([n_cells])

# ax.set_title('CNO+', fontsize=18)

# fig.savefig('/home/sebi/plots/ind/figs4/all_cell_tuning.png', bbox_inches='tight')

FigureCanvasNbAgg()

3242
2941


In [220]:
plt.colorbar(sorted_data)

AttributeError: 'numpy.ndarray' object has no attribute 'autoscale_None'